In [1]:
import pandas as pd
import numpy as np
#for text cleaning
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from tqdm import tqdm_notebook

from nltk.corpus import stopwords

from keras import regularizers, initializers, optimizers, callbacks
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

In [2]:
data = pd.read_csv('output.csv')
data=data.drop(columns='Title')
print(data)
print(data.shape)
# Assuming the sentences are in the first column and the other numerical features are in columns 2 to 11
sentences = data.iloc[:, -1]
numerical_features = data.iloc[0:52]
target = data.iloc[53]
print(sentences)
print(numerical_features)
print(target)

      humidity  temperature  pressure  voc  analog_input  pm2.5_alt  \
0       48.233       66.000  1004.197  NaN          0.01        5.9   
1       46.591       66.533  1003.625  NaN          0.01        5.3   
2       46.867       66.000  1003.603  NaN          0.01        5.8   
3       44.733       64.933  1003.355  NaN          0.01        4.4   
4       43.633       64.934  1003.821  NaN          0.01        4.0   
...        ...          ...       ...  ...           ...        ...   
4663    41.567       81.466  1009.090  NaN          0.01        4.7   
4664    43.900       79.966  1008.723  NaN          0.01        4.5   
4665    47.333       77.967  1008.224  NaN          0.01        4.9   
4666    48.600       76.400  1007.814  NaN          0.01        5.2   
4667    49.292       76.083  1007.466  NaN          0.01        4.7   

      pm2.5_alt_a  pm2.5_alt_b  scattering_coefficient  \
0             5.7          6.1                    21.8   
1             5.1          5.5 

In [3]:
sentences = data.iloc[:, -1].fillna('')
target = data['pm2.5_atm']
numerical_features = data.drop(columns=[data.columns[-1], 'pm2.5_atm'])
numerical_features=numerical_features.fillna(0)
scaler = StandardScaler()
numerical_features = scaler.fit_transform(numerical_features)
print(sentences)

print(numerical_features)
print(target)

0       πατρα πληθος πιστων περιφορα επιταφιου ιερο να...
1                                                        
2                                                        
3                                                        
4                                                        
                              ...                        
4663                                                     
4664                                                     
4665                                                     
4666                                                     
4667                                                     
Name: data_cleaned, Length: 4668, dtype: object
[[ 0.67746038 -0.6500057  -0.56448672 ... -0.03732146 -0.03372048
  -0.24552617]
 [ 0.48348626 -0.58749596 -0.68178657 ... -0.05749429 -0.05280894
  -0.32011797]
 [ 0.51609092 -0.6500057  -0.68629811 ... -0.02423226 -0.0206361
  -0.24423893]
 ...
 [ 0.57114083  0.75347265  0.26132883 ... -0.23791242 -0.233045
  -

In [4]:
print(data['data_cleaned'])

0       πατρα πληθος πιστων περιφορα επιταφιου ιερο να...
1                                                     NaN
2                                                     NaN
3                                                     NaN
4                                                     NaN
                              ...                        
4663                                                  NaN
4664                                                  NaN
4665                                                  NaN
4666                                                  NaN
4667                                                  NaN
Name: data_cleaned, Length: 4668, dtype: object


In [5]:
def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

glove_file = 'el_glove_300d.txt'  # Adjust path if needed
embeddings_index = load_glove_embeddings(glove_file)

# Function to get the GloVe embedding of a sentence
def get_sentence_embedding(sentence, embeddings_index, embedding_dim=300):
    words = word_tokenize(sentence)
    embedding = [embeddings_index[word] for word in words if word in embeddings_index]
    return embedding

# Convert each sentence to its GloVe embedding
embedding_dim = 300
embedded_sentences = [get_sentence_embedding(sentence, embeddings_index, embedding_dim) for sentence in sentences]

# Pad the sequences to ensure all have the same length
max_length = max(len(embedding) for embedding in embedded_sentences)
padded_embeddings = pad_sequences(embedded_sentences, maxlen=max_length, padding='post', dtype='float32')

# Normalize the numerical features
scaler = StandardScaler()
numerical_features_scaled = scaler.fit_transform(numerical_features)

# Flatten the padded embeddings and combine them with the numerical features
flattened_embeddings = padded_embeddings.reshape((padded_embeddings.shape[0], -1))
X = np.hstack((flattened_embeddings, numerical_features_scaled))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(numerical_features, target, test_size=0.2, random_state=42)

In [6]:
# Define the MLP model
from keras.layers import Dropout,Flatten

model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='relu'))  # Assuming a regression problem

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)

# Evaluate the model on the test set
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae}")

Epoch 1/50
47/47 [==============================] - 1s 5ms/step - loss: 115529.6172 - mean_absolute_error: 83.3011 - val_loss: 88629.7266 - val_mean_absolute_error: 70.9529
Epoch 2/50
47/47 [==============================] - 0s 2ms/step - loss: 56929.0938 - mean_absolute_error: 71.2788 - val_loss: 3905.6028 - val_mean_absolute_error: 39.1664
Epoch 3/50
47/47 [==============================] - 0s 2ms/step - loss: 1192.2716 - mean_absolute_error: 20.0936 - val_loss: 344.8161 - val_mean_absolute_error: 12.2227
Epoch 4/50
47/47 [==============================] - 0s 2ms/step - loss: 346.9919 - mean_absolute_error: 11.6892 - val_loss: 190.7281 - val_mean_absolute_error: 9.5871
Epoch 5/50
47/47 [==============================] - 0s 2ms/step - loss: 210.7871 - mean_absolute_error: 9.3546 - val_loss: 137.6217 - val_mean_absolute_error: 8.0261
Epoch 6/50
47/47 [==============================] - 0s 2ms/step - loss: 138.6764 - mean_absolute_error: 7.5773 - val_loss: 97.7067 - val_mean_absolute_err